In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import yaml
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from astropy.table import Table
import astropy.units as u
from scipy.optimize import curve_fit 
from lmfit.models import GaussianModel
plt.ion()

In [ ]:
# This is a class to represent Molecular Clouds and their cores.
class Cloud():

    # instantiate
    def __init__(self, name, configFile=None):

        # Check name
        good_names = ['Perseus', 'MonR2', 'Serpens', 'Ophiuchus']
        if name not in good_names:
            raise ValueError('name must be one of {}'.format(good_names))
        else:
            self.name = name

        # Check configFile and read it in
        if configFile is None:
            self.configFile = 'config.yaml'
        else:
            self.configFile = configFile
        with open(self.configFile) as file:
            self.config = yaml.safe_load(file)

        # Set the data path
        self.dataPath = self.config['CloudConfig'][self.name]['dataPath']
        
        # Set the distance
        self.distance = self.config['CloudConfig'][self.name]['cloudDistancePc']
        
        #Set the feedback map
        self.radiativeFeedbackMap = self.config['CloudConfig'][self.name]['radiativeFeedbackMap']
        
        #Set the core property table
        self.corePropertiesTable = self.config['CloudConfig'][self.name]['corePropertiesTable']
        
        #Set the core table reader
        self.coreTableReader = self.config['CloudConfig'][self.name]['coreTableReader']
        
        #A function reference to coreTableReader
        func = getattr(self, self.coreTableReader, None)
        self.coreTable = func(self.corePropertiesTable)
        
        #Feedback map data
        self.readFeedbackMap()
        
        #Core Locations
        self.getCoreLocations()
        
        #g0 Values
        self.getg0Values()
        
        #g0 Threshold
        self.g0Threshold = self.config['CloudConfig'][self.name]['g0Threshold']
        if self.g0Threshold == 'None':
            self.g0Threshold = self.getg0Thresh()
            
        #Column Density Calculation
        self.calc_column_density()
        
        #Volume Density Calculation
        self.calc_volume_density()
        
        #ISRF Histogram
        self.makeISRFMapHistogram()
        
        #CDF Plot
        self.makeCDF_ISRF()
        
        #Mass Histogram
        self.makeCoreMassHistogram()
        
        #CDF Mass
        self.makeCDF_Mass()
        
        #Area Histogram
        self.makeCoreAreaHistogram()
        
        #CDF Area
        self.makeCDF_Area()
        
        #Column Density Histogram
        self.makeCoreColumnDHistogram()
        
        #CDF Column Density
        self.makeCDF_ColDensity()
    
        #Volume Density Histogram
        self.makeCoreVolumeDHistogram()
        
        #CDF Column Density
        self.makeCDF_VolDensity()
        
        #Add Column Density column to the table
      
            
        # Other members
        # distance to
        # angular size - ignore
        # physical size - ignore
        # radiative feedback (map)
        # number of cores
        # mass of cores
        # location of cores
        # sizes of cores
        # Herschel T map - ignore
        # Herschel NH2 map - ignore
        # YSO locations - ignore

        return
    
    
    # Class Methods
    
    def readSokolTable(self, filename):
        '''A function to read in the Sokol core property table'''
        
        #renaming column names for standardization btwn tables
        sokol = Table.read(filename, format='fits') #astropy table 
        sokol.rename_column('RAJ2000', 'RA')
        sokol.rename_column('DEJ2000', 'DEC')
        sokol.rename_column('Area', 'Area_Footprint') 
        sokol.rename_column('CorrMass', 'MASS')
        
        #converting arcsec2 to pc2, formatting data
        arcsec2_to_pc2 =  u.pc**2/u.arcsec**2
        sokol['AREA'] = sokol['CorrHPPA']
        sokol['AREA'] = self.area_arcsec2_to_parsec2(sokol['AREA'],self.distance)
        sokol['AREA'].format = "{:8.5e}"
        sokol['AREA'] *= arcsec2_to_pc2
        
        #defining new columns for calculations
        return sokol
    
    
    def readEnoch2006Table(self, filename):
        '''A function to read in the Enoch 2006 core property table'''
        
        #renaming column names and setting units
        enoch06 = Table.read(filename, format='fits') #astropy table
        enoch06.rename_column('RAJ2000', 'RA')
        enoch06.rename_column('DEJ2000', 'DEC')
        enoch06['RA'].unit = 'deg'
        enoch06['DEC'].unit = 'deg'
        
        enoch06.rename_column('Mass','MASS')
        MinAxis = enoch06['MinAxis'].data
        MajAxis = enoch06['MajAxis'].data
        Area = MinAxis * MajAxis * 1.1331 * u.arcsec**2 #FWHM**2 to area

        #converting arcsec2 to pc2, formatting data
        enoch06['AREA'] = self.area_arcsec2_to_parsec2(Area,self.distance)
        enoch06['AREA'].format = "{:8.5e}"
        arcsec2_to_pc2 =  u.pc**2/u.arcsec**2 
        enoch06['AREA'] *= arcsec2_to_pc2
        
        return enoch06
    
    
    def readEnoch2008Table(self, filename, CldName): #check for the cloud name
        '''A function to read in the Enoch 2008 core property table for Ophiuchus'''
        
        #renaming column names and setting units
        enoch08 = Table.read(filename, format='fits') #astropy table
        enoch08.rename_column('RAJ2000', 'RA')
        enoch08.rename_column('DEJ2000', 'DEC')
        enoch08['RA'].unit = 'deg'
        enoch08['DEC'].unit = 'deg'
        enoch08.rename_column('TMass','MASS')
        
        Area = 1.1331 * enoch08['theta']**2 * u.arcsec #mult by 1 arcsec (forced units)
        enoch08['AREA'] = self.area_arcsec2_to_parsec2(Area,self.distance)
        enoch08['AREA'].format = "{:8.5e}"
        arcsec2_to_pc2 =  u.pc**2/u.arcsec**2 
        enoch08['AREA'] *= arcsec2_to_pc2
        E08 = enoch08[enoch08['Cld'] == CldName]
        return E08

    def readOphiuchusEnoch2008Table(self, filename):
        return self.readEnoch2008Table(filename,'Oph')
    
    def readSerpensEnoch2008Table(self, filename):
        return self.readEnoch2008Table(filename,'Ser')
       
    
    def area_arcsec2_to_parsec2(self,area_arcsec2, distance_pc):
        '''A function to convert units for area from arcsec2 to pc2'''
        # Convert area to steradians
        area_steradians = area_arcsec2 * (u.arcsec**2).to(u.steradian)
    
        # Convert area to parsecs^2 using the small-angle formula: 
        area_pc2 = area_steradians * distance_pc**2
        return area_pc2
    
    def calc_column_density(self):
        '''This function calculates column density of cores in units 1/cm^2.'''
        
        mass = np.float128(self.coreTable['MASS'])
        area_pc2 = np.float128(self.coreTable['AREA'])
        self.particles = (mass * (2e33) * (6.02e23))/2.8 / u.solMass
        self.area_cm2 = area_pc2.to(u.cm**2)
        col_density = self.particles/self.area_cm2
        
        self.coreTable['Col_Density'] = col_density
        return col_density
    
    
    def calc_volume_density(self):
        '''This function calculates the effective radius and the volume density'''
        
        eff_radius = np.sqrt(self.area_cm2/np.pi)
        volume = (4/3)*(np.pi)*((eff_radius)**3)
        vol_density = (self.particles/volume)
        
        self.coreTable['Vol_Density'] = vol_density
        return vol_density
    
    def readFeedbackMap(self):
        '''A function to read in the feedback map and extract it's data'''
        
        rad_hdu = fits.open(self.radiativeFeedbackMap)
        self.rad_wcs = WCS(rad_hdu[0].header) 
        self.rad_data = np.log10(rad_hdu[0].data)
        
        #print(rad_hdu[0].data.shape,rad_hdu[0].header['NAXIS1'],rad_hdu[0].header['NAXIS2'])
        return
           
    
    def getCoreLocations(self): #must be run after readFeedbackMap
        '''A function to extract core locations from the data'''
        
        ra = self.coreTable['RA']
        dec = self.coreTable['DEC']
        
        c = SkyCoord(ra, dec, frame='icrs',unit="deg")
        xpix, ypix = self.rad_wcs.world_to_pixel(c)
        
        self.coreTable['xpix'] = xpix
        self.coreTable['ypix'] = ypix
        
        return

    def getg0Values(self):
        
        xpix = self.coreTable['xpix']
        ypix = self.coreTable['ypix']
        
        rad_g0 = xpix*0-500 
    
        shape = self.rad_data.shape
        for i in range(len(xpix)):
            if (0<xpix[i]<(shape[1]-1)) & (0<ypix[i]<(shape[0]-1)):  
                tmp = self.rad_data[int(ypix[i]+.5), int(xpix[i]+.5)]

                if ~np.isnan(tmp): 
                    rad_g0[i]=tmp
    
        self.coreTable['rad_g0'] = rad_g0
        return
    
    def getg0Thresh(self): 
        rad_g02 = self.coreTable['rad_g0'][(self.coreTable['rad_g0']!=-500)] 
        g0_threshold = np.median(rad_g02)
        
        return g0_threshold
    
    def makeISRFMapHistogram(self, density=True):
        ''''''
        
        rad_data2 = self.rad_data[~np.isnan(self.rad_data)]
        rad_g02 = self.coreTable['rad_g0'][(self.coreTable['rad_g0']!=-500)]
        
        counts,bins= np.histogram(rad_data2,bins=25, density=density) #all pixel feedback 
        counts1,bins1= np.histogram(rad_g02,bins=25, density=density) #core-sampled feedback
        
        # #Modeling a gaussian to fit the pixel ISRF (bigger sample)
        
        nn = len(counts) #num data points
        def gaussfit(x, a, x0, sigma): 
            return a*np.exp(-(x-x0)**2/(2*sigma**2)) 
        
        param = 3
        guesses = [3,1.7,1]
        
        (a,x0,w),cc = curve_fit(gaussfit,bins[:-1],counts,p0=guesses) #doing the fit, bins was 1 element longer
        (ua,ux0,uw) = 2*np.sqrt(np.diag(cc))
        
        yfit = gaussfit(bins[:-1],a,x0,w)
        yys = (yfit-counts)**2
        chisqr = sum(yys)/(nn-param)
        
        xmod = np.linspace(bins[0],bins[-1],100)
        ymod = gaussfit(xmod,a,x0,w)
        
        print()
        print(f'Reduced chi-squared = {round(chisqr,3)}')
        print(f'a = {round(a,3)} +/- {round(ua,3)}')
        print(f'x0 = {round(x0,3)} +/- {round(ux0,3)}')
        print(f'w = {round(w,3)} +/- {round(uw,3)}')
        print()
        print(f'Standard deviation of {self.dataPath} = {round(np.std(rad_g02),3)}')
        print()
        print(f'Max ISRF of {self.dataPath} = {round(np.max(rad_g02),3)}')
        print()
        print(f'Min ISRF of {self.dataPath} = {round(np.min(rad_g02),3)}')
        
        plt.figure(1)
        plt.clf()
        
        
        plt.stairs(counts, bins, color = '#897317', label=('Radiative Feedback Map'),fill=True, alpha=0.5)
        plt.stairs(counts1, bins1, color = '#925ba0', label=('Cores'),fill=True, alpha=0.5)
        
        plt.plot(xmod,ymod,color = 'tab:blue', label = 'Gaussian Fit to Pixel ISRF')
        plt.axvline(x = self.g0Threshold, color = 'black', label = 'High Feedback Cutoff', linestyle='--')
        
        plt.title('Radiative Feedback and Core Placement for '+ self.dataPath)
        
        if density==True:
            plt.ylabel('Normalized Counts')
        else: 
            plt.ylabel('Counts')
            
        plt.xlabel('ISRF Strength (Log(g$_{0}$))')
        plt.show() 
        plt.legend()
        return
    
    def makeCDF_ISRF(self):
        # Num of data points used 
        rad_g02 = self.coreTable['rad_g0'][(self.coreTable['rad_g0']!=-500)] #core-sampled
        rad_data2 = self.rad_data[~np.isnan(self.rad_data)]
        N1 = len(rad_g02)
        N2 = len(rad_data2)
        # normal distribution 
        data1 = rad_g02
        data2 = rad_data2
  
        # sort the data in ascending order 
        isrf_core = np.sort(data1) 
        isrf_pixel = np.sort(data2) 
        
        # get the cdf values of y 
        norm_rank1 = np.arange(N1) / float(N1) 
        norm_rank2 = np.arange(N2) / float(N2)
        # plotting 
        plt.figure(2)
        plt.clf()
        
        plt.plot(isrf_core, norm_rank1,marker='.',linestyle='-',label = 'Core') 
        plt.plot(isrf_pixel, norm_rank2, marker='.',linestyle='-',label = 'Pixel') 
        plt.xlabel('ISRF Strength (Log(g$_{0}$))') 
        plt.ylabel('Normalized Rank') 
        plt.title('CDF of ISRF Values for ' + self.dataPath) 
        plt.legend()
        plt.show()

        return
  
    def makeCoreMassHistogram(self, density=False):
        '''A function to create a histogram of high vs low radiative feedback core masses'''
        
        massL = np.log10(self.coreTable['MASS'])[(self.coreTable['rad_g0'] <= self.g0Threshold) & (self.coreTable['rad_g0'] != -500)].value
        massH = np.log10(self.coreTable['MASS'])[(self.coreTable['rad_g0'] > self.g0Threshold)].value
        combo = np.concatenate((massL,massH))
        cdata, cbins = np.histogram(combo, bins=10)
        countsH,binsH = np.histogram(massH, bins=cbins, density=density) 
        countsL,binsL = np.histogram(massL, bins=cbins, density=density)
        
        plt.figure(3)
        plt.clf()
        
        plt.stairs(countsH, binsH,color = '#897317',label=('High Feedback Cores Values'),fill=True, alpha=0.5)
        plt.stairs(countsL, binsL,color = '#925ba0',label=('Low Feedback Cores Values'),fill=True, alpha=0.5)
        
        plt.title('Corrected Core Mass for ' + self.dataPath)
        plt.ylabel('Counts')
        plt.xlabel('Log Corrected Mass (M$_{\odot}$)')
        plt.legend(fontsize = 8)
        plt.grid(False)
        plt.show()
        return
    
    def makeCDF_Mass(self):
        # Num of data points used 
        massL = np.log10(self.coreTable['MASS'])[(self.coreTable['rad_g0'] <= self.g0Threshold) & (self.coreTable['rad_g0'] != -500)].value
        massH = np.log10(self.coreTable['MASS'])[(self.coreTable['rad_g0'] > self.g0Threshold)].value
        massAll = np.log10(self.coreTable['MASS'])[(self.coreTable['rad_g0'] != -500)].value
        
        N1 = len(massL)
        N2 = len(massH)
        N3 = len(massAll)
  
        # sort the data in ascending order 
        low_mass = np.sort(massL) 
        high_mass = np.sort(massH)
        all_mass = np.sort(massAll) 
        
        # get the cdf values of y 
        norm_rank1 = np.arange(N1) / float(N1) 
        norm_rank2 = np.arange(N2) / float(N2)
        norm_rank3 = np.arange(N3) / float(N3)
        
        # plotting 
        plt.figure(4)
        plt.clf()
        
        plt.plot(low_mass, norm_rank1, marker='.',linestyle='-',color = '#925ba0', label = 'Low Feedback') 
        plt.plot(high_mass, norm_rank2, marker='.',linestyle='-',color = '#897317',label = 'High Feedback') 
        plt.plot(all_mass, norm_rank3, marker='.',linestyle='-',label = 'All Feedback') 
        plt.xlabel('Log Corrected Mass (M$_{\odot}$)') 
        plt.ylabel('Normalized Rank') 
        plt.title('CDF of Core Mass for ' + self.dataPath) 
        plt.legend()
        plt.show()
        return
    
    def makeCoreAreaHistogram(self, density=False):
        '''A function to create a histogram of high vs low radiative feedback core masses'''
        
        areaL = np.log10(self.coreTable['AREA'])[(self.coreTable['rad_g0'] <= self.g0Threshold) & (self.coreTable['rad_g0'] != -500)].value
        areaH = np.log10(self.coreTable['AREA'])[(self.coreTable['rad_g0'] > self.g0Threshold)].value
        combo = np.concatenate((areaL,areaH))
        cdata, cbins = np.histogram(combo, bins=10)
        countsH,binsH = np.histogram(areaH, bins=cbins, density=density) 
        countsL,binsL = np.histogram(areaL, bins=cbins, density=density)
        
        plt.figure(5)
        plt.clf()
        
        plt.stairs(countsH, binsH,color = '#897317',label=('Area (High Feedback)'),fill=True, alpha=0.5)
        plt.stairs(countsL, binsL,color = '#925ba0',label=('Area (Low Feedback)'),fill=True, alpha=0.5)
        
        plt.title('Area for ' + self.dataPath)
        plt.ylabel('Counts')
        plt.xlabel('Log Area (pc$^{2}$)')
        plt.legend(fontsize = 8)
        plt.grid(False)
        plt.show()
        return
    
    def makeCDF_Area(self):
        # Num of data points used 
        areaL = np.log10(self.coreTable['AREA'])[(self.coreTable['rad_g0'] <= self.g0Threshold) & (self.coreTable['rad_g0'] != -500)].value
        areaH = np.log10(self.coreTable['AREA'])[(self.coreTable['rad_g0'] > self.g0Threshold)].value
        areaAll = np.log10(self.coreTable['AREA'])[(self.coreTable['rad_g0'] != -500)].value
        
        N1 = len(areaL)
        N2 = len(areaH)
        N3 = len(areaAll)
  
        # sort the data in ascending order 
        low_area = np.sort(areaL) 
        high_area = np.sort(areaH)
        all_area = np.sort(areaAll) 
        
        # get the cdf values of y 
        norm_rank1 = np.arange(N1) / float(N1) 
        norm_rank2 = np.arange(N2) / float(N2)
        norm_rank3 = np.arange(N3) / float(N3)
        
        # plotting 
        plt.figure(6)
        plt.clf()
        
        plt.plot(low_area, norm_rank1, marker='.',linestyle='-',color = '#925ba0',label = 'Low Feedback') 
        plt.plot(high_area, norm_rank2, marker='.',linestyle='-',color = '#897317',label = 'High Feedback') 
        plt.plot(all_area, norm_rank3, marker='.',linestyle='-',label = 'All Feedback') 
        plt.xlabel('Log Area (pc$^{2}$)') 
        plt.ylabel('Normalized Rank') 
        plt.title('CDF of Core Area for ' + self.dataPath) 
        plt.legend()
        plt.show()
        return
    
    def makeCoreColumnDHistogram(self, density=False):
        
        colL = np.log10(self.coreTable['Col_Density'])[(self.coreTable['rad_g0'] <= self.g0Threshold) & (self.coreTable['rad_g0'] != -500)].value
        colH = np.log10(self.coreTable['Col_Density'])[(self.coreTable['rad_g0'] > self.g0Threshold)].value
        combo = np.concatenate((colL,colH))
        cdata, cbins = np.histogram(combo, bins=10)
        countsH,binsH = np.histogram(colH,bins=cbins,density=density) 
        countsL,binsL = np.histogram(colL,bins=cbins,density=density)
        
        plt.figure(7)
        plt.clf()
        plt.stairs(countsH, binsH,color = '#897317',label=('Column Density (High Feedback)'),fill=True,alpha=0.5)
        plt.stairs(countsL, binsL,color = '#925ba0',label=('Column Density (Low Feedback)'),fill=True,alpha=0.5)
        plt.ylabel('Counts')
        plt.xlabel('Log Column Density (cm$^{-2}$)')
        plt.title('Column Density for ' + self.dataPath)
        plt.legend(fontsize = 8)
        plt.grid(False)
        plt.show()
        return
    
    def makeCDF_ColDensity(self):
        # Num of data points used 
        colL = np.log10(self.coreTable['Col_Density'])[(self.coreTable['rad_g0'] <= self.g0Threshold) & (self.coreTable['rad_g0'] != -500)].value
        colH = np.log10(self.coreTable['Col_Density'])[(self.coreTable['rad_g0'] > self.g0Threshold)].value
        colAll = np.log10(self.coreTable['Col_Density'])[(self.coreTable['rad_g0'] != -500)].value
        
        N1 = len(colL)
        N2 = len(colH)
        N3 = len(colAll)
  
        # sort the data in ascending order 
        low_col = np.sort(colL) 
        high_col = np.sort(colH)
        all_col = np.sort(colAll) 
        
        # get the cdf values of y 
        norm_rank1 = np.arange(N1) / float(N1) 
        norm_rank2 = np.arange(N2) / float(N2)
        norm_rank3 = np.arange(N3) / float(N3)
        
        # plotting 
        plt.figure(8)
        plt.clf()
        
        plt.plot(low_col, norm_rank1, marker='.',linestyle='-',color = '#925ba0',label = 'Low Feedback') 
        plt.plot(high_col, norm_rank2, marker='.',linestyle='-',color = '#897317',label = 'High Feedback') 
        plt.plot(all_col, norm_rank3, marker='.',linestyle='-',label = 'All Feedback') 
        plt.xlabel('Log Column Density (cm$^{-2}$)') 
        plt.ylabel('Normalized Rank') 
        plt.title('CDF of Core Column Density for ' + self.dataPath) 
        plt.legend()
        plt.show()
        return
    
    def makeCoreVolumeDHistogram(self, density=False):
        
        volL = np.log10(self.coreTable['Vol_Density'])[(self.coreTable['rad_g0'] <= self.g0Threshold) & (self.coreTable['rad_g0'] != -500)].value
        volH = np.log10(self.coreTable['Vol_Density'])[(self.coreTable['rad_g0'] > self.g0Threshold)].value
        combo = np.concatenate((volL,volH))
        cdata, cbins = np.histogram(combo, bins=10)
        countsH,binsH = np.histogram(volH,bins=cbins,density=density) 
        countsL,binsL = np.histogram(volL,bins=cbins,density=density)
        
        plt.figure(9)
        plt.clf()
        plt.stairs(countsH, binsH,color = '#897317',label=('Volume Density (High Feedback)'),fill=True,alpha=0.5)
        plt.stairs(countsL, binsL,color = '#925ba0',label=('Volume Density (Low Feedback)'),fill=True,alpha=0.5)
        plt.ylabel('Counts')
        plt.xlabel('Log Volume Density (cm$^{-3}$)')
        plt.title('Volume Density for ' + self.dataPath)
        plt.legend(fontsize = 8)
        plt.grid(False)
        plt.show()
        return
    
    def makeCDF_VolDensity(self):
        # Num of data points used 
        volL = np.log10(self.coreTable['Vol_Density'])[(self.coreTable['rad_g0'] <= self.g0Threshold) & (self.coreTable['rad_g0'] != -500)].value
        volH = np.log10(self.coreTable['Vol_Density'])[(self.coreTable['rad_g0'] > self.g0Threshold)].value
        volAll = np.log10(self.coreTable['Vol_Density'])[(self.coreTable['rad_g0'] != -500)].value
        
        N1 = len(volL)
        N2 = len(volH)
        N3 = len(volAll)
  
        # sort the data in ascending order 
        low_vol = np.sort(volL) 
        high_vol = np.sort(volH)
        all_vol = np.sort(volAll) 
        
        # get the cdf values of y 
        norm_rank1 = np.arange(N1) / float(N1) 
        norm_rank2 = np.arange(N2) / float(N2)
        norm_rank3 = np.arange(N3) / float(N3)
        
        # plotting 
        plt.figure(10)
        plt.clf()
        
        plt.plot(low_vol, norm_rank1, marker='.',linestyle='-',color = '#925ba0',label = 'Low Feedback') 
        plt.plot(high_vol, norm_rank2, marker='.',linestyle='-',color = '#897317',label = 'High Feedback') 
        plt.plot(all_vol, norm_rank3, marker='.',linestyle='-',label = 'All Feedback') 
        plt.xlabel('Log Volume Density (cm$^{-3}$)') 
        plt.ylabel('Normalized Rank') 
        plt.title('CDF of Core Volume Density for ' + self.dataPath) 
        plt.legend()
        plt.show()
        return

In [ ]:
if __name__ == '__main__':
    p = Cloud('Perseus')
    #o = Cloud('Ophiuchus')
    #s = Cloud('Serpens')
    #m = Cloud('MonR2')
   
    #check whatever you need here

Verbs (Methods)
calculate angular size - ignore
calculate physical size - ignore
read and parse core tables
read Hershel maps - ignore
read in YSO locations - ignore
spatial distribution plot (verify that our masking is correct)

Grave yard
isrf fitting 
xdat = np.array(bins[:-1])
ydat = np.array(counts)
model = GaussianModel()
params = model.guess(ydat, x=xdat)
result = model.fit(ydat, params, x=xdat)
print(result.fit_report())
plt.plot(xdat, result.best_fit, '-', label='best fit',color='red')